
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"

df = spark.read.csv(file_location, header = True, inferSchema = True)

In [0]:
df.show(2)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 2 rows



In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCols=['sex','smoker','day','time'], outputCols=['sex_indexed','smoker_indexed','day_indexed','time_indexed'])
df_r = indexer.fit(df).transform(df)

In [0]:
df_r.show(2)

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
only showing top 2 rows



In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
df_r.columns

Out[16]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
featureassembler = VectorAssembler(inputCols=['tip','size','sex_indexed', 'smoker_indexed', 'day_indexed', 'time_indexed'], outputCol="Independent Features")

In [0]:
output = featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show(3)

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
+--------------------+
only showing top 3 rows



In [0]:
finalized_data = output.select("Independent Features","total_bill")
finalized_data.show(2)

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
+--------------------+----------+
only showing top 2 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = "Independent Features",labelCol = "total_bill")
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[27]: DenseVector([2.9303, 2.9709, -1.1647, 2.0851, 0.022, -2.1259])

In [0]:
regressor.intercept

Out[28]: 3.395944861738581

In [0]:
#predictions
pred_results = regressor.evaluate(test_data)


In [0]:
#prediction comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07| 13.00052783030766|
|(6,[0,1],[1.97,2.0])|     12.02|  15.1103431915794|
| (6,[0,1],[2.5,4.0])|     18.35| 22.60511079887674|
|(6,[0,1],[2.72,2.0])|     13.28| 17.30806752623746|
|(6,[0,1],[4.08,2.0])|     17.92|21.293274319750743|
|(6,[0,1],[6.73,4.0])|     48.27|35.000276046348205|
| (6,[0,1],[9.0,4.0])|     48.33| 41.65205503257994|
|[1.32,2.0,0.0,0.0...|      9.68|13.227678490888511|
|[1.36,3.0,1.0,0.0...|     18.64| 13.04722721096542|
|[1.44,2.0,0.0,0.0...|      7.56|11.475489970441956|
|[1.48,2.0,0.0,0.0...|      8.52|11.592701934957052|
|[1.5,2.0,0.0,0.0,...|     12.46|13.799191776565308|
|[1.5,2.0,0.0,1.0,...|     15.69| 15.84027366368525|
|[1.5,2.0,1.0,0.0,...|     26.41|12.568409239344517|
|[1.56,2.0,0.0,0.0...|      9.94| 13.93095027797909|
|[1.76,2.0,0.0,1.0...|     11.24|16.5801217103

In [0]:
pred_results.r2

Out[33]: 0.6628546728889765

In [0]:
pred_results.meanAbsoluteError

Out[34]: 4.66917559507718

In [0]:
pred_results.meanSquaredError

Out[35]: 37.01534329410139

In [0]:
(pred_results.meanSquaredError)**(0.5)

Out[36]: 6.084023610580534

In [0]:
#end